# Wholesale Pricing Logic

This notebook handles wholesale pricing calculations and uploads for different regions.

## Table of Contents
1. [Setup & Dependencies](#setup)
2. [Configuration](#config)
3. [Helper Functions](#functions)
4. [Data Loading](#data-loading)
5. [Data Processing](#data-processing)
6. [Export & Upload](#export-upload)


---
## 1. Setup & Dependencies <a id='setup'></a>


In [1]:
%%capture

# =============================================================================
# Package Installation
# =============================================================================

# Core
!pip install --upgrade pip

# Database Connectivity
!pip install psycopg2-binary
!pip install snowflake-connector-python==3.15.0
!pip install snowflake-sqlalchemy
!pip install sqlalchemy==1.4.46

# AWS & API
!pip install boto3
!pip install requests
!pip install keyring==23.11.0

# Google Sheets
!pip install oauth2client
!pip install gspread==5.9.0
!pip install gspread_dataframe
!pip install google.cloud

# Data Manipulation
!pip install pandas==2.2.1
!pip install numpy
!pip install polars
!pip install openpyxl
!pip install xlsxwriter

# Utilities
!pip install tqdm
!pip install warnings
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
!pip install db-dtypes
!pip install import-ipynb

# Analytics
!pip install statsmodels
!pip install scikit-learn
!pip install pulp
!pip install slack_sdk
!pip install slack
!pip install aiohttp 

In [2]:
# =============================================================================
# Imports
# =============================================================================

# Standard Library
import os
import json
import time
import base64
import warnings
import calendar
import importlib
from pathlib import Path
from datetime import date, datetime, timedelta

# Data Processing
import numpy as np
import pandas as pd

# Database
import snowflake.connector

# AWS
import boto3
from botocore.exceptions import ClientError

# HTTP & API
import requests
from requests import get

# Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Utilities
from tqdm import tqdm
import import_ipynb

# Custom Environment Setup
import setup_environment_2

# =============================================================================
# Configuration
# =============================================================================
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()

/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron


/home/ec2-user/service_account_key.json


---
## 2. Configuration <a id='config'></a>


In [3]:
# =============================================================================
# Constants & Mappings
# =============================================================================

# All cohort IDs used in queries
ALL_COHORT_IDS = [700, 701, 702, 703, 704, 696, 695, 698, 697, 699, 1123, 1124, 1125, 1126]

# Warehouse mappings (region, warehouse_name, warehouse_id, cohort_id)
WAREHOUSE_MAPPING = [
    ('Cairo', 'Mostorod', 1, 700),
    ('Giza', 'Barageel', 236, 701),
    ('Delta West', 'El-Mahala', 337, 703),
    ('Delta West', 'Tanta', 8, 703),
    ('Delta East', 'Mansoura FC', 339, 704),
    ('Delta East', 'Sharqya', 170, 704),
    ('Upper Egypt', 'Assiut FC', 501, 1124),
    ('Upper Egypt', 'Bani sweif', 401, 1126),
    ('Upper Egypt', 'Menya Samalot', 703, 1123),
    ('Upper Egypt', 'Sohag', 632, 1125),
    ('Alexandria', 'Khorshed Alex', 797, 702),
    ('Giza', 'Sakkarah', 962, 701)
]

# Region to Cohort mapping for uploads
REGION_COHORT_MAPPING = {
    'Greater Cairo': 1156,
    'Upper Egypt': 1190,
    'Delta': 1222,
    'Alexandria': 1223
}

# Tier-based buffer for price calculations
TIER_BUFFER_MAP = {
    1: 0.7,
    2: 0.75,
    3: 0.8,
    4: 0.85
}

# Products to exclude from upload
EXCLUDED_PRODUCT_IDS = [4541, 12973]

# Brands with special handling
ADDITIONAL_BRANDS_REDUCE = ['هارفست فوودز', 'البوادي']
BRANDS_TO_REMOVE_FROM_REDUCE = ['فيوري']
# Upload chunk sizes
CHUNK_SIZE_DEFAULT = 4000
CHUNK_SIZE_SPECIAL = 2000  # For cohort 61


---
## 3. Helper Functions <a id='functions'></a>


In [4]:
# =============================================================================
# Database Functions
# =============================================================================

def query_snowflake(query: str, columns: list = None) -> pd.DataFrame:
    """
    Execute a query on Snowflake and return results as DataFrame.
    
    Args:
        query: SQL query string to execute
        columns: Optional list of column names for the result DataFrame
    
    Returns:
        DataFrame with query results
    """
    columns = columns or []
    
    con = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USERNAME"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        database=os.environ["SNOWFLAKE_DATABASE"]
    )
    
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        
        data = np.array(cur.fetchall())
        if len(columns) == 0:
            return pd.DataFrame(data)
        return pd.DataFrame(data, columns=columns)
    
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    
    finally:
        cur.close()
        con.close()


def get_snowflake_timezone() -> str:
    """Get the current timezone setting from Snowflake."""
    query = "SHOW PARAMETERS LIKE 'TIMEZONE'"
    result = query_snowflake(query)
    return result[1].values[0]


def convert_columns_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert all columns to numeric where possible.
    
    Args:
        df: Input DataFrame
    
    Returns:
        DataFrame with numeric columns where conversion was possible
    """
    df.columns = df.columns.str.lower()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

In [5]:
# =============================================================================
# AWS & API Functions
# =============================================================================

def get_secret(secret_name: str) -> str:
    """
    Retrieve a secret from AWS Secrets Manager.
    
    Args:
        secret_name: Name of the secret to retrieve
    
    Returns:
        Secret string or decoded binary
    """
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_messages = {
            'DecryptionFailureException': "Can't decrypt secret using provided KMS key",
            'InternalServiceErrorException': "Server-side error occurred",
            'InvalidParameterException': "Invalid parameter value provided",
            'InvalidRequestException': "Invalid request for current resource state",
            'ResourceNotFoundException': "Requested resource not found"
        }
        if error_code in error_messages:
            print(f"AWS Error: {error_messages[error_code]}")
        raise e
    
    if 'SecretString' in response:
        return response['SecretString']
    return base64.b64decode(response['SecretBinary'])

In [6]:
def get_access_token(url: str, client_id: str, client_secret: str) -> str:
    """
    Get OAuth access token for MaxAB API.
    
    Args:
        url: Token endpoint URL
        client_id: OAuth client ID
        client_secret: OAuth client secret
    
    Returns:
        Access token string
    """
    response = requests.post(
        url,
        data={
            "grant_type": "password",
            "username": API_USERNAME,
            "password": API_PASSWORD
        },
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def _get_api_token() -> str:
    """Get fresh API token for MaxAB requests."""
    return get_access_token(
        'https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
        'main-system-externals',
        API_SECRET
    )


def post_prices(cohort_id: int, file_name: str) -> requests.Response:
    """
    Upload pricing sheet to MaxAB API for a specific cohort.
    
    Args:
        cohort_id: Target cohort ID
        file_name: Path to Excel file with pricing data
    
    Returns:
        API response object
    """
    token = _get_api_token()
    url = f"https://api.maxab.info/main-system/api/admin-portal/cohorts/{cohort_id}/pricing"
    
    files = [('sheet', (file_name, open(file_name, 'rb'), 
              'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))]
    headers = {'Authorization': f'bearer {token}'}
    
    return requests.post(url, headers=headers, data={}, files=files)


def post_cart_rules(cohort_id: int, file_name: str) -> requests.Response:
    """
    Upload cart rules sheet to MaxAB API for a specific cohort.
    
    Args:
        cohort_id: Target cohort ID
        file_name: Path to Excel file with cart rules
    
    Returns:
        API response object
    """
    token = _get_api_token()
    url = f"https://api.maxab.info/main-system/api/admin-portal/cohorts/{cohort_id}/cart-rules"
    
    files = [('sheet', (file_name, open(file_name, 'rb'),
              'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))]
    headers = {'Authorization': f'bearer {token}'}
    
    return requests.post(url, headers=headers, data={}, files=files)

In [7]:
# =============================================================================
# Pricing Logic Functions
# =============================================================================

# Minimum margin floor (1%)
MIN_MARGIN = -0.05

def calculate_final_wac(row: pd.Series) -> float:
    """
    Calculate final WAC (Weighted Average Cost) considering price ups.
    """
    if pd.isna(row['new_pp']):
        return row['wac_p']
    else:
        diff = (row['new_pp']-row['wac1']) / row['wac1']
        if diff > 0:
            wac_p = ((0.3*diff)+1)*row['wac_p']
        else:
            wac_p = row['wac_p']  
        return wac_p


def select_price(row: pd.Series) -> float:
    """
    Calculate the wholesale price based on various factors including:
    - Brand-specific rules
    - Category-specific rules  
    - Tier-based pricing
    - Value-to-weight status
    - Minimum margin floor of 1%
    
    Args:
        row: DataFrame row with product and pricing data
    
    Returns:
        Calculated wholesale price
    """
    # Calculate base WAC
    final_wac = calculate_final_wac(row)
    # Adjust target margin for price ups
    target_margin = row['target_margin']
    
    # Brand-specific pricing rules
    if row['brand'] in FORCED_BRAND_LIST:
        if row['brand'] in ['كوكا كولا', 'شويبس']:
            margin_factor = np.maximum(row['margin'] * 0.75, MIN_MARGIN)
            return np.minimum(final_wac / np.maximum((1 - margin_factor), (0.25 * target_margin)),row['price']*0.995)
        elif row['brand'] == 'جود كير':
            margin_factor = np.maximum(row['margin'], MIN_MARGIN)
            return np.minimum(final_wac / np.maximum((1 - margin_factor), (0.5 * target_margin)),row['price'])
        elif row['brand'] == 'بيتي عصاير':
            margin_factor = np.maximum(row['margin'] * 0.9, MIN_MARGIN)
            return np.minimum(final_wac / np.maximum((1 - margin_factor), (0.85 * target_margin)),row['price']*0.995)    
        else:
            margin_factor = np.maximum(np.maximum(row['margin'] * 0.85, MIN_MARGIN),target_margin * 0.85)
            return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    
    # Special brand handling
    if row['brand'] == 'فيوري':
        margin_factor = np.maximum(row['margin'] * 0.9, MIN_MARGIN)
        return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    if row['brand'] in ['فريش فارم','ماى ميت','ستار']:
        margin_factor = np.maximum(row['margin'], MIN_MARGIN)
        return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    
    # Category-specific rules
    if row['cat'] in ['ورقيات','حفاضات أطفال']:
        margin_factor = np.maximum(np.minimum(np.maximum(0.6 * target_margin, 0.015), target_margin), MIN_MARGIN)
        return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    if row['brand'] == 'ريد بل':
        margin_factor = np.maximum(np.minimum(np.maximum(0.5 * target_margin, 0.015), target_margin), MIN_MARGIN)
        return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    # Tier-based pricing with VTW adjustment
    vtw_multiplier = 1 if row['vtw_status'] else 1.3
    
    tier_configs = {
        1: (0.20, 0.0125),
        2: (0.25, 0.015),
        3: (0.40, 0.0175),
        4: (0.60, 0.0175)
    }
    
    tier = row['tier']
    if tier in tier_configs:
        factor, min_tier_margin = tier_configs[tier]
        margin_factor = np.minimum(
        np.minimum(
            np.maximum((factor * vtw_multiplier) * target_margin, min_tier_margin),
            target_margin
        ),
        row['margin']*0.75
        )
        # Ensure minimum 1% margin
        margin_factor = np.maximum(margin_factor, MIN_MARGIN)
        return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)
    
    # Default case - ensure minimum 1% margin
    margin_factor = np.maximum(target_margin, MIN_MARGIN)
    return np.minimum(final_wac / (1 - margin_factor),row['price']*0.995)


In [8]:
# =============================================================================
# Initialize API Credentials & Timezone
# =============================================================================

# Load API credentials
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
API_USERNAME = pricing_api_secret["egypt_username"]
API_PASSWORD = pricing_api_secret["egypt_password"]
API_SECRET = pricing_api_secret["egypt_secret"]

# Get Snowflake timezone
TIMEZONE = get_snowflake_timezone()
print(f"Using timezone: {TIMEZONE}")


Using timezone: America/Los_Angeles


---
## 4. Data Loading <a id='data-loading'></a>


In [9]:
# =============================================================================
# Load SKU Prices
# =============================================================================

cohort_ids_str = ','.join(map(str, ALL_COHORT_IDS))

prices_query = f'''
WITH skus_prices AS (
    WITH local_prices AS (
        SELECT  
            CASE 
                WHEN cpu.cohort_id IN (700, 695) THEN 'Cairo'
                WHEN cpu.cohort_id IN (701) THEN 'Giza'
                WHEN cpu.cohort_id IN (704, 698) THEN 'Delta East'
                WHEN cpu.cohort_id IN (703, 697) THEN 'Delta West'
                WHEN cpu.cohort_id IN (696, 1123, 1124, 1125, 1126) THEN 'Upper Egypt'
                WHEN cpu.cohort_id IN (702, 699) THEN 'Alexandria'
            END AS region,
            cohort_id,
            pu.product_id,
            pu.packing_unit_id,
            pu.basic_unit_count,
            AVG(cpu.price) AS price
        FROM cohort_product_packing_units cpu
        JOIN PACKING_UNIT_PRODUCTS pu ON pu.id = cpu.product_packing_unit_id
        WHERE cpu.cohort_id IN ({cohort_ids_str})
            AND cpu.created_at::date <> '2023-07-31'
            AND cpu.is_customized = TRUE
        GROUP BY ALL
    ),
    
    live_prices AS (
        SELECT 
            region, cohort_id, product_id, 
            pu_id AS packing_unit_id, 
            buc AS basic_unit_count, 
            NEW_PRICE AS price
        FROM materialized_views.DBDP_PRICES
        WHERE created_at = CURRENT_DATE
            AND DATE_PART('hour', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())) 
                BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND (SPLIT_PART(time_slot, '-', 1)::int) + 1
            AND cohort_id IN ({cohort_ids_str})
    ),
    
    prices AS (
        SELECT *
        FROM (
            SELECT *, 1 AS priority FROM live_prices
            UNION ALL
            SELECT *, 2 AS priority FROM local_prices
        )
        QUALIFY ROW_NUMBER() OVER (PARTITION BY region, cohort_id, product_id, packing_unit_id ORDER BY priority) = 1
    )
    
    SELECT region, cohort_id, product_id, price
    FROM prices
    WHERE basic_unit_count = 1
        AND ((product_id = 1309 AND packing_unit_id = 2) OR (product_id <> 1309))
)

SELECT distinct
    region, cohort_id, p.product_id,
    CONCAT(products.name_ar, ' ', products.size, ' ', product_units.name_ar) AS sku,
    b.name_ar AS brand,
    cat.name_ar AS cat,
    wac1, wac_p, p.price
FROM skus_prices p
JOIN finance.all_cogs c ON c.product_id = p.product_id 
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) BETWEEN c.from_date AND c.to_date
JOIN products ON products.id = p.product_id
JOIN categories cat ON cat.id = products.category_id
JOIN brands b ON b.id = products.brand_id
JOIN product_units ON product_units.id = products.unit_id
WHERE wac1 > 0 AND wac_p > 0
GROUP BY ALL
'''

whole_sale = query_snowflake(
    prices_query, 
    columns=['region', 'cohort_id', 'product_id', 'sku', 'brand', 'cat', 'wac1', 'wac_p', 'price']
)
whole_sale = convert_columns_to_numeric(whole_sale).drop_duplicates()

print(f"Loaded {len(whole_sale):,} wholesale price records")

Loaded 102,758 wholesale price records


In [10]:
whole_sale = whole_sale.groupby(['region', 'cohort_id', 'product_id', 'sku', 'brand', 'cat', 'price']).agg({'wac1':'max','wac_p':'max'}).reset_index()

In [11]:
# =============================================================================
# Load Margin Targets
# =============================================================================

# Brand-Category level targets
brand_target_query = '''
SELECT DISTINCT cat, brand, margin AS target_bm
FROM performance.commercial_targets cplan
QUALIFY 
    CASE 
        WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CURRENT_DATE) 
        THEN DATE_TRUNC('month', CURRENT_DATE)
        ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') 
    END = DATE_TRUNC('month', date)
'''

brand_cat_target = query_snowflake(brand_target_query, columns=['cat', 'brand', 'target_bm'])
brand_cat_target['target_bm'] = pd.to_numeric(brand_cat_target['target_bm'])

# Category level targets (weighted average)
cat_target_query = '''
SELECT cat, SUM(target_bm * (target_nmv / cat_total)) AS cat_target_margin
FROM (
    SELECT *, SUM(target_nmv) OVER(PARTITION BY cat) AS cat_total
    FROM (
        SELECT cat, brand, AVG(target_bm) AS target_bm, SUM(target_nmv) AS target_nmv
        FROM (
            SELECT DISTINCT date, city AS region, cat, brand, margin AS target_bm, nmv AS target_nmv
            FROM performance.commercial_targets cplan
            QUALIFY 
                CASE 
                    WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CURRENT_DATE) 
                    THEN DATE_TRUNC('month', CURRENT_DATE)
                    ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') 
                END = DATE_TRUNC('month', date)
        )
        GROUP BY ALL
    )
)
GROUP BY ALL
'''

cat_target = query_snowflake(cat_target_query, columns=['cat', 'cat_target_margin'])
cat_target['cat_target_margin'] = pd.to_numeric(cat_target['cat_target_margin'])

print(f"Loaded {len(brand_cat_target):,} brand targets and {len(cat_target):,} category targets")

Loaded 471 brand targets and 76 category targets


In [12]:
# =============================================================================
# Load Sales Data
# =============================================================================

sales_query = '''
with cohort_data as (
SELECT DISTINCT
    cpc.cohort_id,
    pso.product_id,
    SUM(pso.total_price) AS nmv
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN COHORT_PRICING_CHANGES cpc ON cpc.id = pso.COHORT_PRICING_CHANGE_ID and cpc.cohort_id  in (1156,1190,1222,1223)
WHERE so.created_at::date BETWEEN  CURRENT_DATE -60 AND CURRENT_DATE-1
    AND so.sales_order_status_id NOT IN (7, 12)
    AND so.channel IN ('telesales', 'retailer')
    AND pso.purchased_item_count <> 0
GROUP BY ALL
)
select cohort_id_pricing as cohort_id,product_id,nmv 
from (
select cd.*, c.FALLBACK_COHORT_ID,c2.id as cohort_id_pricing
from cohort_data cd 
join cohorts c on c.id = cd.cohort_id
join cohorts c2 on c2.FALLBACK_COHORT_ID = c.FALLBACK_COHORT_ID and c2.id in (700,701,702,703,704,1123,1124,1125,1126)
)
'''

sales = query_snowflake(sales_query, columns=['cohort_id', 'product_id', 'nmv'])
sales = convert_columns_to_numeric(sales)

print(f"Loaded {len(sales):,} sales records")    

Loaded 9,128 sales records


In [13]:
# =============================================================================
# Load Reference Data (Groups, Packing Units, Region Mapping)
# =============================================================================

# Commercial Groups
groups_query = 'SELECT * FROM materialized_views.sku_commercial_groups'
groups = setup_environment_2.dwh_pg_query(groups_query, columns=['product_id', 'group'])
groups = convert_columns_to_numeric(groups)

print(f"Loaded {len(groups):,} commercial groups")      
    

Loaded 1,601 commercial groups


In [14]:
# =============================================================================
# Load Stock Data
# =============================================================================

# Build warehouse values from config
whs_values = ', '.join([f"('{r}', '{w}', {wid}, {cid})" for r, w, wid, cid in WAREHOUSE_MAPPING])

stocks_query = f'''
WITH whs AS (
    SELECT * FROM (VALUES {whs_values}) x(region, wh, warehouse_id, cohort_id)
)

SELECT cohort_id, product_id, SUM(stocks) AS stocks
FROM (
    SELECT DISTINCT 
        whs.region,
        cohort_id,
        whs.wh,
        product_warehouse.product_id,
        product_warehouse.available_stock::integer AS stocks
    FROM whs
    JOIN product_warehouse ON product_warehouse.warehouse_id = whs.warehouse_id
    JOIN products ON product_warehouse.product_id = products.id
    JOIN product_units ON products.unit_id = product_units.id
    WHERE product_warehouse.warehouse_id NOT IN (6, 9, 10)
        AND product_warehouse.is_basic_unit = 1
        AND product_warehouse.available_stock > 0
)
GROUP BY ALL
'''

stocks = query_snowflake(stocks_query, columns=['cohort_id', 'product_id', 'stocks'])
stocks = convert_columns_to_numeric(stocks)

print(f"Loaded {len(stocks):,} stock records")     

Loaded 14,778 stock records


In [15]:
# Packing Units
pu_query = '''
SELECT product_id, PACKING_UNIT_id, basic_unit_count
FROM PACKING_UNIT_PRODUCTS
WHERE deleted_at IS NULL
ORDER BY product_id, basic_unit_count
'''

pu = query_snowflake(pu_query, columns=['product_id', 'pu_id', 'buc'])
pu = convert_columns_to_numeric(pu)

print(f"Loaded {len(pu):,} packing units")

Loaded 35,312 packing units


In [16]:
# Price Ups
price_ups_query = '''
SELECT product_id, new_pp, forecasted_date
FROM materialized_views.DBDP_PRICE_UPS
WHERE region = 'Cairo'
'''

price_ups = query_snowflake(price_ups_query, columns=['product_id', 'new_pp', 'forcasted_date'])
price_ups = convert_columns_to_numeric(price_ups)

print(f"Loaded {len(price_ups):,} price up records")        

Loaded 142 price up records


In [17]:
# Region Mapping
region_query = '''
SELECT DISTINCT 
    CASE WHEN r.name_en LIKE '%Delta%' THEN 'Delta' ELSE r.name_en END AS main_region,
    CASE WHEN r.id = 2 THEN s.name_en ELSE r.name_en END AS region
FROM regions r
JOIN states s ON s.region_id = r.id
'''

region_mapping = query_snowflake(region_query, columns=['main_region', 'region'])
region_mapping.columns = region_mapping.columns.str.lower()

print(f"Loaded {len(region_mapping):,} region mappings")

Loaded 7 region mappings


In [18]:
# =============================================================================
# Load Value-to-Weight (VTW) Data
# =============================================================================

vtw_query = '''
WITH whs AS (
    SELECT * FROM (VALUES
        ('Cairo', 'El-Marg', 38), ('Cairo', 'Mostorod', 1),
        ('Giza', 'Barageel', 236), ('Giza', 'Basatin', 39),
        ('Delta West', 'El-Mahala', 337), ('Delta West', 'Tanta', 8),
        ('Delta East', 'Mansoura FC', 339), ('Delta East', 'Sharqya', 170),
        ('Upper Egypt', 'Assiut FC', 501), ('Upper Egypt', 'Bani sweif', 401),
        ('Upper Egypt', 'Menya Samalot', 703), ('Upper Egypt', 'Sohag', 632),
        ('Alexandria', 'Khorshed Alex', 797)
    ) x(region, wh, warehouse_id)
),

region_vtw AS (
    SELECT region, nmv/weight AS r_vtw
    FROM (
        SELECT whs.region, 
            SUM(product_sales_order.total_price) AS nmv,
            SUM((packing_unit_products.weight * product_sales_order.PURCHASED_ITEM_COUNT) / 1000.00) AS weight
        FROM sales_orders
        JOIN product_sales_order ON product_sales_order.sales_order_id = sales_orders.id
        JOIN whs ON whs.warehouse_id = product_sales_order.warehouse_id
        JOIN packing_unit_products ON product_sales_order.product_id = packing_unit_products.product_id 
            AND product_sales_order.packing_unit_id = packing_unit_products.packing_unit_id
        JOIN products ON products.id = product_sales_order.product_id
        JOIN packing_units ON packing_units.id = product_sales_order.packing_unit_id
        JOIN product_units ON product_units.id = products.unit_id
        JOIN categories ON categories.id = products.category_id
        JOIN sections ON categories.section_id = sections.id
        JOIN brands ON brands.id = products.brand_id
        WHERE sales_orders.CREATED_AT::date >= CURRENT_DATE - 30
            AND sales_orders.sales_order_status_id NOT IN (7, 12)
        GROUP BY ALL
    )
),

product_vtw AS (
    SELECT region, product_id, SUM(p_vtw * cntrb) AS p_vtw
    FROM (
        SELECT *, nmv / SUM(nmv) OVER(PARTITION BY region, product_id) AS cntrb
        FROM (
            SELECT region, product_id, packing_unit_id, nmv, nmv/weight AS p_vtw
            FROM (
                SELECT whs.region, product_sales_order.product_id, product_sales_order.packing_unit_id,
                    SUM(product_sales_order.total_price) AS nmv,
                    SUM((packing_unit_products.weight * product_sales_order.PURCHASED_ITEM_COUNT) / 1000.00) AS weight
                FROM sales_orders
                JOIN product_sales_order ON product_sales_order.sales_order_id = sales_orders.id
                JOIN whs ON whs.warehouse_id = product_sales_order.warehouse_id
                JOIN packing_unit_products ON product_sales_order.product_id = packing_unit_products.product_id 
                    AND product_sales_order.packing_unit_id = packing_unit_products.packing_unit_id
                JOIN products ON products.id = product_sales_order.product_id
                JOIN packing_units ON packing_units.id = product_sales_order.packing_unit_id
                JOIN product_units ON product_units.id = products.unit_id
                JOIN categories ON categories.id = products.category_id
                JOIN sections ON categories.section_id = sections.id
                JOIN brands ON brands.id = products.brand_id
                WHERE sales_orders.CREATED_AT::date >= CURRENT_DATE - 30
                    AND sales_orders.sales_order_status_id NOT IN (7, 12)
                GROUP BY ALL
            )
            WHERE weight > 0
        )
    )
    GROUP BY ALL
)

SELECT pv.*, rv.r_vtw
FROM product_vtw pv
JOIN region_vtw rv ON rv.region = pv.region
'''

vtw = query_snowflake(vtw_query, columns=['region', 'product_id', 'p_vtw', 'r_vtw'])
vtw = convert_columns_to_numeric(vtw)

print(f"Loaded {len(vtw):,} VTW records")         

Loaded 13,831 VTW records


In [19]:
# =============================================================================
# Load Google Sheets Data (Brands & Categories Overrides)
# =============================================================================

# Initialize Google Sheets client
GSHEET_SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_dict(
    json.loads(setup_environment_2.get_secret("prod/maxab-sheets")), 
    GSHEET_SCOPE
)
client = gspread.authorize(creds)

# Load campaign brands for reduction
brands_list = client.open('Anniversary Campaign 2025 (Final)').worksheet('Suppliers Brands')
brands_df = pd.DataFrame(brands_list.get_all_records())[['Brands']].drop_duplicates()
brands_reduce = list(brands_df['Brands']) + ADDITIONAL_BRANDS_REDUCE

for brand in BRANDS_TO_REMOVE_FROM_REDUCE:
    if brand in brands_reduce:
        brands_reduce.remove(brand)

print(f"Loaded {len(brands_reduce)} brands for reduction")

Loaded 127 brands for reduction


In [20]:
def to_numeric_columns(df):
    """Convert all columns to numeric where possible."""
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

def convert_sku_id(row):
    """Convert SKU string to integer ID."""
    try:
        return int(str(row.SKU).replace(",", ""))
    except:
        return row.SKU

In [21]:
# Load forced brands and categories from execution sheet
force_brands_ws = client.open('Wholesales_exec').worksheet('brands')
force_cats_ws = client.open('Wholesales_exec').worksheet('cats')
force_cart_rules = client.open('Wholesales_exec').worksheet('Cart rules')

force_brands_df = pd.DataFrame(force_brands_ws.get_all_records())
force_cats_df = pd.DataFrame(force_cats_ws.get_all_records())
force_cart_rules = pd.DataFrame(force_cart_rules.get_all_records())

FORCED_BRAND_LIST = list(force_brands_df.brand.unique()) if not force_brands_df.empty else []
FORCED_CAT_LIST = list(force_cats_df.cat.unique()) if not force_cats_df.empty else []

print(f"Forced brands: {len(FORCED_BRAND_LIST)}, Forced categories: {len(FORCED_CAT_LIST)}")

Forced brands: 14, Forced categories: 1


---
## 5. Data Processing <a id='data-processing'></a>
        

In [22]:
# =============================================================================
# Merge All Data Sources & Calculate Base Metrics
# =============================================================================

# Merge targets
wholesale_data = whole_sale.merge(brand_cat_target, on=['cat', 'brand'], how='left')
wholesale_data = wholesale_data.merge(cat_target, on=['cat'], how='left')

# Merge operational data
wholesale_data = wholesale_data.merge(stocks, on=['product_id', 'cohort_id'], how='left')
wholesale_data = wholesale_data.merge(vtw, on=['product_id', 'region'], how='left')
wholesale_data = wholesale_data.fillna(0)

# Calculate VTW status and margins
wholesale_data['vtw_status'] = wholesale_data['p_vtw'] >= wholesale_data['r_vtw']
wholesale_data['stocks'] = wholesale_data['stocks'].fillna(0)
wholesale_data['margin'] = (wholesale_data['price'] - wholesale_data['wac_p']) / wholesale_data['price']
wholesale_data['target_margin'] = (
    wholesale_data['target_bm']
    .fillna(wholesale_data['cat_target_margin'])
    .fillna(wholesale_data['margin'])
)

# Merge sales and region mapping
wholesale_data = wholesale_data.merge(sales, on=['product_id', 'cohort_id'], how='left')
wholesale_data['nmv'] = wholesale_data['nmv'].fillna(0)
wholesale_data = region_mapping.merge(wholesale_data, on=['region'])

# Aggregate to main region level
aggregation = {'price': 'mean', 'stocks': 'sum', 'margin': 'mean', 'nmv': 'sum', 'vtw_status': 'max'}
wholesale_data = wholesale_data.groupby(
    ['main_region', 'product_id', 'sku', 'brand', 'cat', 'target_margin', 'wac1', 'wac_p']
).agg(aggregation).reset_index()

# Merge price ups and calculate contribution
wholesale_data = wholesale_data.merge(price_ups, on=['product_id'], how='left')
wholesale_data['total_nmv'] = wholesale_data.groupby('main_region')['nmv'].transform('sum')
wholesale_data['cntrb'] = wholesale_data['nmv'] / wholesale_data['total_nmv']
wholesale_data = wholesale_data.sort_values(['main_region', 'nmv'], ascending=[True, False])
wholesale_data['nmv_cumulative_cntrb'] = wholesale_data.groupby('main_region')['cntrb'].cumsum()

print(f"Processed data shape: {wholesale_data.shape}")

cond = [wholesale_data['nmv_cumulative_cntrb'] < 0.3 ,
        (wholesale_data['nmv_cumulative_cntrb'] >= 0.3)&(wholesale_data['nmv_cumulative_cntrb'] < 0.6),
        (wholesale_data['nmv_cumulative_cntrb'] >= 0.6)&(wholesale_data['nmv_cumulative_cntrb'] < 0.8),
        wholesale_data['nmv_cumulative_cntrb'] >= 0.8
       ] 
cho = [1,2,3,4]

wholesale_data['tier'] = np.select(cond,cho,default = 4)

# wholesale_data.loc[wholesale_data['brand'].isin(brands_reduce),'tier']=np.maximum(wholesale_data['tier']-1,1)
# wholesale_data.loc[wholesale_data['brand'].isin(ADDITIONAL_BRANDS_REDUCE),'tier']=1
#wholesale_data=wholesale_data.merge(tgtg_long[['main_region','product_id','TGTG_f']],on=['main_region','product_id'],how='left')
#wholesale_data.loc[~wholesale_data['TGTG_f'].isna(),'tier']=1
wholesale_data['base_price'] =  wholesale_data.apply(select_price,axis=1)
wholesale_data['new_margin'] =  (wholesale_data['base_price']-wholesale_data['wac_p'])/wholesale_data['base_price']
wholesale_data['drop_margin'] = ((wholesale_data['new_margin']-wholesale_data['margin'])/wholesale_data['margin'])*-1
buffer_map = {
    1: 0.7,   
    2: 0.7,  
    3: 0.7,   
    4: 0.7  
}
# You can change these numbers to be more/less aggressive.

wholesale_data['buffer_B'] = wholesale_data['tier'].map(buffer_map)
wholesale_data['allowed_discount_fraction'] = wholesale_data['margin'] * wholesale_data['buffer_B']
wholesale_data['wholesale_min_price'] = wholesale_data['price'] * (1 - wholesale_data['allowed_discount_fraction'])
wholesale_data['min_margin'] = (wholesale_data['wholesale_min_price'] -wholesale_data['wac_p']) /wholesale_data['wholesale_min_price']  
wholesale_data['selected_margin'] = np.maximum(wholesale_data['min_margin'],wholesale_data['new_margin'])
wholesale_data['selected_price'] = wholesale_data['wac_p']/(1-wholesale_data['selected_margin'])

Processed data shape: (30482, 18)


In [23]:
# =============================================================================
# Apply Group-Based Pricing
# =============================================================================

wholesale_data['price_diff'] = (wholesale_data['selected_price'] - wholesale_data['price']) / wholesale_data['price']
wholesale_data = wholesale_data.merge(groups, on=['product_id'], how='left')

# Calculate group-weighted prices
wholesale_data['new_group_nmv'] = wholesale_data['nmv']
wholesale_data.loc[wholesale_data['stocks'] == 0, 'new_group_nmv'] = wholesale_data['nmv'] * 0.1

wholesale_data['total_group_nmv'] = wholesale_data.groupby(['main_region', 'group'])['new_group_nmv'].transform('sum')
wholesale_data['price_cntrb'] = wholesale_data['selected_price'] * (wholesale_data['new_group_nmv'] / wholesale_data['total_group_nmv'])
wholesale_data['final_group_price'] = wholesale_data.groupby(['main_region', 'group'])['price_cntrb'].transform('sum')

# Set final price with fallbacks
wholesale_data['final_price'] = wholesale_data['selected_price']
wholesale_data['final_price'] = np.ceil(wholesale_data['final_price'] * 4) / 4  # Round to nearest 0.25

# Handle edge cases
wholesale_data.loc[wholesale_data['final_price'] == 0, 'final_price'] = wholesale_data['selected_price']
wholesale_data.loc[wholesale_data['final_price'] == 0, 'final_price'] = wholesale_data['price']

In [24]:
# Apply forced category pricing (keep original price for specified categories)
wholesale_data.loc[wholesale_data['cat'].isin(FORCED_CAT_LIST), 'final_price'] = wholesale_data['price']

print(f"Final pricing complete. Records: {len(wholesale_data):,}") 

Final pricing complete. Records: 30,482


In [25]:
# =============================================================================
# Export Full Data to Excel
# =============================================================================

wholesale_data.to_excel('Wholesales_new_price_list.xlsx', index=False)
print("Exported: Wholesales_new_price_list.xlsx")

Exported: Wholesales_new_price_list.xlsx


---
## 6. Export & Upload <a id='export-upload'></a>


In [26]:
# =============================================================================
# Prepare Upload Data
# =============================================================================

# Prepare final data with packing units
final_data = wholesale_data[['main_region', 'product_id', 'sku', 'brand', 'cat','wac_p', 'final_price', 'tier']]
final_data = final_data.drop_duplicates()
final_data = final_data.merge(pu, on='product_id')
final_data['new_price'] = final_data['final_price'] * final_data['buc']
final_data['wac_p'] = final_data['wac_p'] * final_data['buc']
final_data['margin'] = (final_data['new_price']-final_data['wac_p'])/final_data['new_price']

# Handle packing unit indexing
final_data['ind'] = 1
final_data['ind'] = final_data.groupby(['main_region', 'product_id']).ind.cumsum()

# Load and apply minimum PU removal rules
remove_min_pu = pd.read_csv('skus_to_remove_min.csv')
remove_min_pu['remove_min'] = 1
final_data = final_data.merge(remove_min_pu[['product_id', 'remove_min']], on='product_id', how='left')

final_data['max_ind'] = final_data.groupby(['product_id', 'main_region'])['ind'].transform('max')
final_data.loc[(final_data['max_ind'] > 1) & (final_data['ind'] == 1), 'remove_min'] = 1

print(f"Upload data prepared. Records: {len(final_data):,}")

Upload data prepared. Records: 45,691


In [27]:
# =============================================================================
# Prepare Cart Rules Data
# =============================================================================

cart_rules_data = final_data[final_data['new_price'] > 0].copy()

# Calculate allowed quantities based on tier
cart_rules_data['half_allowed_quantity'] = 25000 / cart_rules_data['new_price']
cart_rules_data.loc[cart_rules_data['tier'] == 1, 'half_allowed_quantity'] = 20000 / cart_rules_data['new_price']
cart_rules_data.loc[cart_rules_data['margin'] <= 0, 'half_allowed_quantity'] = 10000 / cart_rules_data['new_price']

cart_rules_data['Cart_rules'] = np.ceil(cart_rules_data['half_allowed_quantity'])

# Brand-specific overrides
cart_rules_data.loc[cart_rules_data['brand'].isin(['بست', 'فيوري']), 'Cart_rules'] = 10

print(f"Cart rules prepared. Records: {len(cart_rules_data):,}")

Cart rules prepared. Records: 45,691


In [28]:
# =============================================================================
# Finalize Upload DataFrames
# =============================================================================

# Prepare price upload data
to_upload = final_data[['product_id', 'sku', 'pu_id', 'new_price', 'main_region', 'ind', 'remove_min']]
to_upload = to_upload.drop_duplicates()
to_upload = to_upload.dropna(subset=['new_price'])
to_upload = to_upload[(to_upload['new_price'] > 1)].reset_index(drop=True)

# Region to cohort mapping (using config)
mapping_cc = pd.DataFrame(REGION_COHORT_MAPPING.items(), columns=['main_region', 'new_cohort_id'])

In [29]:
# Apply cohort mapping
to_upload = to_upload.merge(mapping_cc, on='main_region')
cart_rules_data = cart_rules_data.merge(mapping_cc, on='main_region')

to_upload['cohort_id'] = to_upload['new_cohort_id']
cart_rules_data['cohort_id'] = cart_rules_data['new_cohort_id']

to_upload = to_upload.drop(columns=['new_cohort_id', 'main_region'])
cart_rules_data = cart_rules_data.drop(columns=['new_cohort_id', 'main_region'])

In [30]:
# Exclude specific products
for pid in EXCLUDED_PRODUCT_IDS:
    to_upload = to_upload[to_upload['product_id'] != pid]
    cart_rules_data = cart_rules_data[cart_rules_data['product_id'] != pid]

# Finalize cart rules columns
cart_rules_data = cart_rules_data[['cohort_id', 'product_id', 'pu_id', 'Cart_rules']]

print(f"To upload: {len(to_upload):,} price records")
print(f"Cart rules: {len(cart_rules_data):,} records")
print(f"Cohorts: {to_upload.cohort_id.unique()}")

To upload: 45,671 price records
Cart rules: 45,679 records
Cohorts: [1223 1222 1156 1190]


In [31]:
# rem_min = pd.read_csv('skus_to_remove_min.csv')
# to_upload = pd.read_excel('Fixed_prices.xlsx')
# to_upload['remove_min'] = np.nan
# to_upload.loc[to_upload['product_id'].isin(rem_min.product_id.unique()), 'remove_min']=1
# to_upload = to_upload[to_upload['product_id']!=11794]
# to_upload=to_upload[~to_upload['new_price'].isna()]
# df_mapping = pd.DataFrame({
#     "cohort_id": [700, 701, 702, 703, 704, 1123],
#     "new_cohort": [61, 695, 699, 697, 698, 696]
# })

# to_upload_ext = to_upload.merge(df_mapping,on='cohort_id')
# to_upload_ext=to_upload_ext.drop(columns = 'cohort_id')
# to_upload_ext=to_upload_ext.rename(columns = {'new_cohort':'cohort_id'})
# to_upload = pd.concat([to_upload,to_upload_ext])

In [32]:
# =============================================================================
# Upload Prices to API
# =============================================================================

for cohort in to_upload.cohort_id.unique():
    print(f"\n{'='*50}")
    print(f"Processing cohort: {cohort}")
    print('='*50)
    
    # Prepare cohort data
    upload = to_upload[to_upload['cohort_id'] == cohort].copy()
    out = upload[['product_id', 'sku', 'pu_id', 'new_price', 'ind', 'remove_min']].copy()
    out.columns = ['Product ID', 'Product Name', 'Packing Unit ID', 'Price', 'ind', 'remove_min']
    
    # Set visibility
    out['Visibility (YES/NO)'] = 'YES'
    out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
    out = out.drop(columns=['ind', 'remove_min']).drop_duplicates()
    
    # Add required columns
    out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
    out['Tags'] = None
    
    # Save full file
    file_name_ = f'WS_uploads/1_new_{cohort}.xlsx'.replace(' ', '_')
    out.to_excel(file_name_, index=False, engine='xlsxwriter')
    time.sleep(5)
    
    # Split into chunks for upload
    chunk_size = CHUNK_SIZE_SPECIAL if cohort == 61 else CHUNK_SIZE_DEFAULT
    chunks = [out[i:i + chunk_size] for i in range(0, len(out), chunk_size)]
    print(f"Split into {len(chunks)} chunks (size: {chunk_size})")
    
    # Save chunk files
    fileslist = []
    for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Saving chunks"):
        output_file = f'WS_uploads/manual/output_{cohort}_chunk_{i + 1}.xlsx'
        fileslist.append(output_file)
        chunk.to_excel(output_file, index=False, engine='xlsxwriter')
    
    # Upload chunks
    print("Uploading...")
    upload_success = True
    
    for file in fileslist:
        chunk_num = file.split('chunk_')[1].split('.xls')[0]
        response = post_prices(cohort, file)
        
        if '"success":true' in str(response.content).lower():
            print(f"  ✓ Chunk {chunk_num} uploaded successfully")
        else:
            print(f"  ✗ ERROR chunk {chunk_num}")
            print(f"    Response: {response.content}")
            upload_success = False
            break
    
    if not upload_success:
        print(f"Upload failed for cohort {cohort}. Stopping.")
        break

print("\nPrice upload complete!")


Processing cohort: 1223


Split into 3 chunks (size: 4000)


Saving chunks:   0%|          | 0/3 [00:00<?, ?it/s]

Saving chunks:  33%|███▎      | 1/3 [00:00<00:00,  3.87it/s]

Saving chunks:  67%|██████▋   | 2/3 [00:00<00:00,  3.86it/s]

Saving chunks: 100%|██████████| 3/3 [00:00<00:00,  4.14it/s]

Saving chunks: 100%|██████████| 3/3 [00:00<00:00,  4.05it/s]

Uploading...


  ✗ ERROR chunk 1
    Response: b'{"success":false,"message":"Product ID, Packing Unit ID in rows 1145, 1146 are equal!, Product ID, Packing Unit ID in rows 1147, 1148 are equal!, Product ID, Packing Unit ID in rows 1401, 1402 are equal!, Product ID, Packing Unit ID in rows 1403, 1404 are equal!, Product ID, Packing Unit ID in rows 1405, 1406 are equal!, Product ID, Packing Unit ID in rows 1407, 1408 are equal!, Product ID, Packing Unit ID in rows 1409, 1410 are equal!, Product ID, Packing Unit ID in rows 1411, 1412 are equal!, Product ID, Packing Unit ID in rows 1660, 1661 are equal!, Product ID, Packing Unit ID in rows 1662, 1663 are equal!, Product ID, Packing Unit ID in rows 3021, 3022 are equal!, Product ID, Packing Unit ID in rows 3023, 3024 are equal!, Product ID, Packing Unit ID in rows 3025, 3026 are equal!, Product ID, Packing Unit ID in rows 3027, 3028 are equal!, Product ID, Packing Unit ID in rows 3029, 3030 are equal!, Product ID, Packing Unit ID in rows 3031, 3032 are eq

In [33]:
# =============================================================================
# Upload Cart Rules to API
# =============================================================================

print("\nUploading Cart Rules...")
print('='*50)

for cohort in cart_rules_data.cohort_id.unique():
    req_data = cart_rules_data[cart_rules_data['cohort_id'] == cohort]
    
    if len(req_data) == 0:
        print(f"  No cart rules for cohort {cohort}")
        continue
    
    # Prepare and save cart rules file
    req_data = req_data[['product_id', 'pu_id', 'Cart_rules']]
    req_data.columns = ['Product ID', 'Packing Unit ID', 'Cart Rules']
    
    file_name = f'WS_uploads/Cart_rules/CartRules_{cohort}.xlsx'
    req_data.to_excel(file_name, index=False, engine='xlsxwriter')
    time.sleep(5)
    
    # Upload
    response = post_cart_rules(cohort, file_name)
    
    if response.ok:
        print(f"  ✓ Cohort {cohort}: Cart rules uploaded successfully")
    else:
        print(f"  ✗ ERROR cohort {cohort}")
        print(f"    Response: {response.content}")
        break

print("\nCart rules upload complete!")


Uploading Cart Rules...


  ✗ ERROR cohort 1223
    Response: b'{"status":false,"message":"Product ID, Packing Unit ID in rows 1145, 1146 are equal!, Product ID, Packing Unit ID in rows 1147, 1148 are equal!, Product ID, Packing Unit ID in rows 1401, 1402 are equal!, Product ID, Packing Unit ID in rows 1403, 1404 are equal!, Product ID, Packing Unit ID in rows 1405, 1406 are equal!, Product ID, Packing Unit ID in rows 1407, 1408 are equal!, Product ID, Packing Unit ID in rows 1409, 1410 are equal!, Product ID, Packing Unit ID in rows 1411, 1412 are equal!, Product ID, Packing Unit ID in rows 1660, 1661 are equal!, Product ID, Packing Unit ID in rows 1662, 1663 are equal!, Product ID, Packing Unit ID in rows 3021, 3022 are equal!, Product ID, Packing Unit ID in rows 3023, 3024 are equal!, Product ID, Packing Unit ID in rows 3025, 3026 are equal!, Product ID, Packing Unit ID in rows 3027, 3028 are equal!, Product ID, Packing Unit ID in rows 3029, 3030 are equal!, Product ID, Packing Unit ID in rows 3031, 3032 are